Link dos datasets

Samba: https://www.kaggle.com/datasets/clovesgtx/brazilian-music-samba-lyrics

Gospel: https://raw.githubusercontent.com/damarals/letras/master/inst/csv/letras.csv

Axe: https://www.kaggle.com/datasets/jorgefjr/brazilian-songs-lyrics

Funk: https://www.kaggle.com/datasets/sleshes/funk-carioca-dataset

Bossa Nova: https://www.kaggle.com/datasets/mcarujo/bossa-nova-lyrics

Rock,Sertanejo, Samba2 = https://github.com/gabriellmd/data-frames-nlp

In [2]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [3]:
bossa = pd.read_csv('datasets/bossa_nova_songs_dataset.csv')
samba2 = pd.read_csv('datasets/musicas_samba.csv')
sertanejo = pd.read_csv('datasets/musicas_sertanejo.csv')
samba = pd.read_csv('datasets/samba_dataset.csv',sep='|')
axe = pd.read_csv('datasets/letras_mus_br_axe.csv')
funk = pd.read_csv('datasets/Dataset_funk_carioca.csv')
rock = pd.read_csv('datasets/musicas_rock.csv') 
gospel = pd.read_csv('datasets/gospel.csv') 

Padronização dos datasets 

In [4]:
gospel_ajustado = pd.DataFrame(gospel['letra'])  #OK
gospel_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK

In [5]:
rock_ajustado = pd.DataFrame(rock['Lyric'])  #OK
rock_ajustado.rename(columns={'Lyric': 'lyric'}, inplace = True) #OK

In [6]:
samba_ajustado = pd.DataFrame(samba['letra'])  #OK
samba_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK

In [7]:
samba2_ajustado = pd.DataFrame(samba2['Lyric'])  #OK
samba2_ajustado.rename(columns={'Lyric': 'lyric'}, inplace = True) #OK

In [8]:
axe_ajustado = pd.DataFrame(axe['letras'])  #OK
axe_ajustado.rename(columns={'letras': 'lyric'}, inplace = True) #OK

In [9]:
funk_ajustado = pd.DataFrame(funk['letra'])  #OK
funk_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK

In [10]:
bossa_ajustado = pd.DataFrame(bossa['song_lyrics'])  #OK
bossa_ajustado.rename(columns={'song_lyrics': 'lyric'}, inplace = True) #OK

In [11]:
sertanejo_ajustado = pd.DataFrame(sertanejo['Lyric'])  #OK
sertanejo_ajustado.rename(columns={'Lyric': 'lyric'}, inplace = True) #OK

Criando a coluna correspondete ao genero

In [12]:
bossa_ajustado['gen'] = 'Bossa Nova'
funk_ajustado['gen'] = 'Funk'
gospel_ajustado['gen'] = 'Gospel'
sertanejo_ajustado['gen'] = 'Sertanejo'
axe_ajustado['gen'] = 'Axe'
rock_ajustado['gen'] = 'Rock'
samba_ajustado['gen'] = 'Samba'
samba2_ajustado['gen'] = 'Samba'

In [13]:
datasets = [bossa_ajustado,funk_ajustado,gospel_ajustado,sertanejo_ajustado,axe_ajustado,rock_ajustado,samba_ajustado,samba2_ajustado]

In [14]:
lyrics = pd.concat(datasets)

# Dataset final 

In [15]:
lyrics.reset_index(drop=True,inplace=True)
lyrics

,lyric,gen
0,"De tudo, ao meu amor serei atento antes E co...",Bossa Nova
1,Era uma casa Muito engraçada Não tinha teto ...,Bossa Nova
2,E por falar em saudade Onde anda você Onde a...,Bossa Nova
3,Filhos... Filhos? Melhor não tê-los! Mas se ...,Bossa Nova
4,É melhor ser alegre que ser triste Alegria é...,Bossa Nova
...,...,...
65762,Tom............:. Introdução.: Dm C#o Gm C7 F ...,Samba
65763,"Resolvi te abandonar te deixar para traz, deci...",Samba
65764,Ela mexe comigo. E o pior que não sabe. Coment...,Samba
65765,O pai me disse que a tradição é lanterna. Vem ...,Samba


# Pré-processamento

In [21]:
def processamento_texto(txt):
    # remover quebras de linha
    txt = txt.replace('\n',' ')
    # remover símbolos de pontuação, resultando em um array de caracteres
    txt = [char for char in txt if char not in string.punctuation]
    # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
    txt = ''.join(txt).split()
    # por fim, remover as stopwords da lista
    #txt = [word for word in txt if word.lower() not in stopwords.words('portuguese')]
    
    return txt

## Naive-Bayes

In [100]:
naive_bayes_array = np.array([])

In [ ]:
from sklearn.model_selection import cross_validate

for count in range(5):
    lyric_train,lyric_test,gen_train,gen_test = train_test_split(lyrics['lyric'],lyrics['gen'],test_size=0.3)
    pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('nb',MultinomialNB())
    ])
    pipeline.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    scoring = {'acc': 'accuracy',
           'prec_macro': 'precision_macro',
           'rec_micro': 'recall_macro'}
    scores = cross_validate(pipeline,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    naive_bayes_array = np.append(naive_bayes_array, scores)
media_nb = naive_bayes_array.mean()
media_nb

C:\Users\Lucas Araújo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lucas Araújo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lucas Araújo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

## Random Forest

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [65]:
#rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [72]:
def convert(data):
    number = preprocessing.LabelEncoder()
    data['lyric'] = number.fit_transform(data.lyric)
    #data['gen'] = number.fit_transform(data.gen)
    data=data.fillna(-999)
    return data

lyric_train.column #= convert(lyric_train)
#gen_train = convert(gen_train)

AttributeError: 'Series' object has no attribute 'column'

In [74]:
pipeline_2 = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('rf',RandomForestClassifier())
])
#train=convert(train)
#test=convert(test)
#pipeline_2.fit(lyric_train,gen_train)

In [75]:
pipeline_2.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function processamento_texto at 0x0000016E723D5870>,
                                 encoding='unicode')),
                ('tfidf', TfidfTransformer()),
                ('rf', RandomForestClassifier())])

In [76]:
predictions_2 = pipeline_2.predict(lyric_test.values.astype('U'))

In [77]:
print(classification_report(gen_test,predictions_2))

C:\Users\Lucas Araújo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         Axe       0.74      0.06      0.11       280
  Bossa Nova       0.89      0.35      0.50      1802
        Funk       0.93      0.15      0.26       328
      Gospel       0.87      1.00      0.93     16353
        Rock       0.00      0.00      0.00         3
       Samba       0.81      0.13      0.22       959
   Sertanejo       0.00      0.00      0.00         6

    accuracy                           0.87     19731
   macro avg       0.60      0.24      0.29     19731
weighted avg       0.87      0.87      0.83     19731



C:\Users\Lucas Araújo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lucas Araújo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
